# Capstone lab - Week 3

### Step 1: Load in the data

Data from [wikipedia](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) scraped into Excel using the CTRL+C and CTRL+V approach

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('toronto_neighborhoods.xlsx', sheet_name='Data')
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
df.drop(df[df['Borough']=='Not assigned'].index, axis=0, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
# Confirm that postal codes are already unique
print(df['Postal Code'].value_counts())

df[df['Postal Code'] == 'M5A']

M7A    1
M4C    1
M1G    1
M1C    1
M1M    1
      ..
M9B    1
M9L    1
M8W    1
M1X    1
M4E    1
Name: Postal Code, Length: 103, dtype: int64


,Postal Code,Borough,Neighborhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
# Check if there are any rows without a neighborhood
sum(df['Neighborhood'] == 'Not assigned')

0

In [6]:
# Print out the number of rows in the dataframe
df.reset_index()
df.shape[0]

103

### Step 2: Geocode the neighborhoods

In [7]:
import geocoder

postal_codes = df['Postal Code'].values
lats = []
lons = []

try:
    for postal_code in postal_codes:

        lat_lng_coords = None
        while(lat_lng_coords is None):
            g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))

            if str(g) == '<[REQUEST_DENIED] Google - Geocode [empty]>':
                raise Exception()

            lat_lng_coords = g.latlng

        if lat_lng_coords is not None:    
            lats.append(lat_lng_coords[0])
            lons.append(lat_lng_coords[1])

    df["Latitude"] = lats
    df["Longitude"] = lons

    df.head()
    
except Exception:
    print("Geocoding API not working")

Geocoding API not working


In [8]:
# Use the CSV instead

df_latlon = pd.read_csv("Geospatial_Coordinates.csv")
df_latlon.head()

df = df.merge(df_latlon, on='Postal Code')
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
# Filter only to boroughs that contain "Toronto" in them for simplicity

df = df[["Toronto" in borough for borough in df['Borough']]]
df = df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


### Step 3: Load venue data from Foursquare

In [10]:
import folium

In [11]:
map_toronto = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [12]:
# Load Foursquare credentials
import json

with open("foursquare_api_key.json", "r") as key_file:
    key = json.load(key_file)
    
CLIENT_ID = key['CLIENT_ID']
CLIENT_SECRET = key['CLIENT_SECRET']
VERSION = '20180605'

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
# Load in the top 100 venues in each neighborhood
use_excel = True

if use_excel:
    df_venues = pd.read_excel("toronto_venues.xlsx", sheet_name='Data')
    
    print("Loaded rows from Excel:", df_venues.shape[0])
    
else:
    import requests

    LIMIT = 100
    radius = 500

    # Initialize an empty list for each column
    neigh = []
    neigh_lat = []
    neigh_lon = []
    venue = []
    venue_lat = []
    venue_lon = []
    venue_cat = []

    for lat, lon, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lon, 
            radius, 
            LIMIT)

        results = requests.get(url).json()['response']['groups'][0]['items']

        for v in results:
            neigh.append(neighborhood)
            neigh_lat.append(lat)
            neigh_lon.append(lon)

            venue.append(v['venue']['name'])
            venue_lat.append(v['venue']['location']['lat'])
            venue_lon.append(v['venue']['location']['lng'])  
            venue_cat.append(v['venue']['categories'][0]['name'])

        print("Loaded venues so far:", len(venue))

    df_venues = pd.DataFrame({
        'Neighborhood':neigh,
        'Neighborhood Latitude':neigh_lat,
        'Neighborhood Longitude':neigh_lon,
        'Venue':venue,
        'Venue Latitude':venue_lat,
        'Venue Longitude':venue_lon,
        'Venue Category':venue_cat
        })
    
    df_venues.to_excel("toronto_venues.xlsx", sheet_name='Data', index=False)    

Loaded rows from Excel: 1606


In [20]:
df_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


### Step 4: Analyze venue data at neighborhood level

In [31]:
df_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")
df_onehot['Neighborhood'] = df_venues['Neighborhood']

neigh_column = list(df_onehot.columns).index('Neighborhood')
fixed_columns = [df_onehot.columns[neigh_column]] + list(df_onehot.columns[:neigh_column]) + list(df_onehot.columns[neigh_column+1:])
df_onehot = df_onehot[fixed_columns]

df_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
df_onehot.shape

(1606, 233)

In [52]:
# Print the top 5 categories for a few neighborhoods
num_top_venues = 5

df_grouped = df_onehot.groupby('Neighborhood').mean().reset_index()

for hood in df_grouped['Neighborhood'][0:5]:
    print("Neighborhood:", hood)
    
    temp = df_grouped[df_grouped['Neighborhood'] == hood].T.reset_index().iloc[1:]
    temp.columns = ['venue','freq']
    
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Neighborhood: Berczy Park
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2      Beer Bar  0.04
3    Restaurant  0.04
4          Café  0.04


Neighborhood: Brockton, Parkdale Village, Exhibition Place
                   venue  freq
0                   Café  0.13
1         Breakfast Spot  0.09
2  Performing Arts Venue  0.09
3            Coffee Shop  0.09
4                 Bakery  0.04


Neighborhood: Business reply mail Processing Centre
           venue  freq
0    Yoga Studio  0.06
1  Garden Center  0.06
2           Park  0.06
3     Comic Shop  0.06
4     Restaurant  0.06


Neighborhood: CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4           Airport  0.06


Neighborhood: Central Bay Street
                 venue  freq
0          Coffee Shop  0.17
1   Italian Restaurant  0.06
2 

In [85]:
# Create a new dataframe with the top 10 venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Seafood Restaurant,Bakery,Restaurant,Cheese Shop,Beer Bar,Diner,Japanese Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Performing Arts Venue,Coffee Shop,Grocery Store,Restaurant,Bar,Intersection,Bakery,Stadium
2,Business reply mail Processing Centre,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop,Light Rail Station
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boat or Ferry,Rental Car Location,Sculpture Garden,Boutique,Plane,Airport Food Court
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Restaurant,Ice Cream Shop,Bar,Salad Place,Thai Restaurant


### Step 5: Run the clustering

In [86]:
from sklearn.cluster import KMeans

kclusters = 7

df_grouped_clustering = df_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

kmeans.labels_[0:10]

array([1, 0, 1, 1, 1, 0, 1, 1, 1, 3], dtype=int32)

In [87]:
# Attach the result to the original data

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df
df_merged = df_merged.merge(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Theater,Breakfast Spot,Pub,Café,Electronics Store,Beer Store,Spa
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Arts & Crafts Store,Burrito Place,Café,Smoothie Shop,Japanese Restaurant,Italian Restaurant,Beer Bar,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Italian Restaurant,Café,Japanese Restaurant,Bubble Tea Shop,Restaurant,Cosmetics Shop,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Seafood Restaurant,Beer Bar,Creperie,Italian Restaurant,Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Asian Restaurant,Health Food Store,Pub,Trail,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Department Store,Donut Shop


In [88]:
# And visualize the clusters on a map
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[df_merged['Latitude'].mean(), df_merged['Longitude'].mean()], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Step 6: Examine the clusters

In [89]:
cluster_num = 0
df_merged.loc[df_merged['Cluster Labels'] == cluster_num, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]].head(10)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,Grocery Store,Café,Park,Restaurant,Italian Restaurant,Coffee Shop,Nightclub,Candy Store,Baby Store,Diner
14,West Toronto,0,Café,Breakfast Spot,Performing Arts Venue,Coffee Shop,Grocery Store,Restaurant,Bar,Intersection,Bakery,Stadium


In [90]:
cluster_num = 1
df_merged.loc[df_merged['Cluster Labels'] == cluster_num, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Park,Bakery,Theater,Breakfast Spot,Pub,Café,Electronics Store,Beer Store,Spa
1,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Arts & Crafts Store,Burrito Place,Café,Smoothie Shop,Japanese Restaurant,Italian Restaurant,Beer Bar,Distribution Center
2,Downtown Toronto,1,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Italian Restaurant,Café,Japanese Restaurant,Bubble Tea Shop,Restaurant,Cosmetics Shop,Ramen Restaurant
3,Downtown Toronto,1,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Seafood Restaurant,Beer Bar,Creperie,Italian Restaurant,Restaurant
4,East Toronto,1,Asian Restaurant,Health Food Store,Pub,Trail,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Department Store,Donut Shop
5,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Café,Seafood Restaurant,Bakery,Restaurant,Cheese Shop,Beer Bar,Diner,Japanese Restaurant
6,Downtown Toronto,1,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Restaurant,Ice Cream Shop,Bar,Salad Place,Thai Restaurant
8,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Clothing Store,Deli / Bodega,Gym,Thai Restaurant,Hotel,Bookstore,Seafood Restaurant
9,West Toronto,1,Pharmacy,Bakery,Coffee Shop,Bar,Supermarket,Bank,Brewery,Café,Portuguese Restaurant,Pool
10,Downtown Toronto,1,Coffee Shop,Aquarium,Café,Hotel,Italian Restaurant,Fried Chicken Joint,Scenic Lookout,Restaurant,Brewery,Sporting Goods Shop


In [91]:
cluster_num = 2
df_merged.loc[df_merged['Cluster Labels'] == cluster_num, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
21,Central Toronto,2,Park,Trail,Jewelry Store,Sushi Restaurant,Bus Line,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [92]:
cluster_num = 3
df_merged.loc[df_merged['Cluster Labels'] == cluster_num, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Central Toronto,3,Gym / Fitness Center,Food & Drink Shop,Department Store,Dance Studio,Hotel,Sandwich Place,Breakfast Spot,Park,Ethiopian Restaurant,Event Space


In [93]:
cluster_num = 4
df_merged.loc[df_merged['Cluster Labels'] == cluster_num, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Garden,Home Service,Ice Cream Shop,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [94]:
cluster_num = 5
df_merged.loc[df_merged['Cluster Labels'] == cluster_num, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Downtown Toronto,5,Park,Playground,Trail,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
